In [ ]:
import re
import sqlite3
import datetime
import glob

# Create the Database 

In [ ]:
#create the local database
db = sqlite3.connect("northstar_athletics.db")

In [ ]:
#set the cursor
cur = db.cursor()

In [ ]:
#Create the Table

#DROP TABLE IF EXISTS is used to avoid an error if we try to drop the table and it doesn't exist
cur.execute('''DROP TABLE IF EXISTS transactions''')
#create table "transactions" and define columns and datatypes
cur.execute('''CREATE TABLE transactions (
    date TEXT, 
    ticket_num INT,
    emp_num INT, 
    cust TEXT, 
    sku TEXT, 
    item TEXT, 
    size TEXT, 
    quan_sold INT, 
    quan_returned INT, 
    retail_price FLOAT, 
    price FLOAT,
    unmatched FLOAT, 
    perks FLOAT, 
    markdown FLOAT, 
    returned FLOAT, 
    net_sale FLOAT, 
    cost FLOAT, 
    gp_percent INT,
    time TEXT
    )''')

db.commit()

# Write Transaction Rows to Database

## Set the path to the Sales Journal files

In [ ]:
#Set the path to the folder containing all the sales journal files 
path = "Sales Journal Clean/*.txt"

#Check to make sure it's reading all the file names
for fname in glob.glob(path) :
    print(fname)

## Define Regular Expressions

In [ ]:
start_of_new_ticket = re.compile(r'D[ ]{1,}[0-9]{3,5}')
good_row_re = re.compile(r'D[ ]{7}[0-9]')
date_re = re.compile(r'1?[0-9]/[0-9]{1,2}/[0-9]{4}')
time_re = re.compile(r'[0-9]{2}:[0-9]{2} [AP][M]')


## Loop to Write Transaction Rows to Database

In [ ]:
for fname in glob.glob(path) :    
    
    hit_start = False
    new_ticket = False
    have_date = False

    with open(fname, 'r') as ifile: 
        for row in ifile :

            if not have_date and date_re.search(row) :
                the_date = date_re.search(row).group(0)
                the_date = datetime.datetime.strptime(the_date, "%m/%d/%Y").strftime("%Y-%m-%d")
                have_date = True

            if hit_start :
                if start_of_new_ticket.search(row) and not new_ticket and "Regular" in row:
                    if "Ticket VOIDED" not in row:
                        new_ticket = True

                        output_rows = []


                        ticket_num = row[1:7].strip()
                        emp_num = row[8:10]
                        cust = row[25:100].strip()

                elif new_ticket :
                    # is ticket over? 
                    if  row.startswith("NP") or "-"*47 in row :
                        new_ticket = False

                        # Get time
                        if "-"*47 in row : 
                            the_time = time_re.search(row).group(0) 
                        else : 
                            the_time = ""

                        # add rows to the database
                        for o_row in output_rows:
                            o_row.append(the_time)
                            
                            cur.execute('''
                            INSERT INTO transactions (date,ticket_num,emp_num,cust,sku,item,size,quan_sold,quan_returned,
                                      retail_price,price,unmatched,perks,markdown,returned,net_sale,cost,gp_percent,time)
                            VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)''', o_row)
                           
                    else :
                        # if not over, process row
                        if good_row_re.search(row) and "Ticket VOIDED" not in row:
                            if "Totals" not in row:
                                sku = row[13:29].strip()
                                item = row[29:50].strip()
                                size = row[50:62].strip()
                                quan_sold = row[62:67].strip()
                                quan_returned = row[67:71].strip()
                                retail_price = row[71:80].strip()
                                price = row[80:90].strip()
                                unmatched = row[90:99].strip()
                                perks = row[99:110].strip()
                                markdown = row[110:120].strip()
                                returned = row[120:130].strip()
                                net_sale = row[130:141].strip()
                                cost = row[141:150].strip()
                                gp_percent = row[150:156].strip()



                            output_rows.append([the_date,ticket_num,emp_num,cust,sku,item,size,quan_sold,quan_returned,
                                          retail_price,price,unmatched,perks,markdown,returned,net_sale,cost,gp_percent])


            if "-------- SKU --------" in row :
                hit_start = True

db.commit()